# EVIDENCE DETECTION : SOLUTION B (*Deep learning-based approaches that do not employ transformer architectures*)



---



## **TRAINING ON TRAIN DATASET**

The following code is used to train the model.

In [1]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report


In [3]:
#Load data
train_data = pd.read_csv('train.csv')
dev_data = pd.read_csv('dev.csv')

# Prepare the data - concatenate 'Claim' and 'Evidence' as the input text
X_train = train_data['Claim'] + " " + train_data['Evidence']
y_train = train_data['label']


In [4]:
import pickle

# Tokenize the text
max_words = 10000

# Tokenizer fitting
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Save the tokenizer to a file
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)


In [5]:
# Pad sequences to a fixed length
max_len = 100  #this cover 100% of the dataset
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)


In [6]:
# Build the Bidirectional LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Single output node for binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [7]:
dev_data = pd.read_csv('dev.csv')
X_dev = dev_data['Claim'] + " " + dev_data['Evidence']
y_dev = dev_data['label']

# Tokenize and pad sequences for the development data
# use the tokenizer that was fit on the training data
X_dev_seq = tokenizer.texts_to_sequences(X_dev)
X_dev_pad = pad_sequences(X_dev_seq, maxlen=max_len)  # Ensure max_len is the same as for the training data

In [8]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model on the entire training dataset
model.fit(X_train_pad, y_train, epochs=10, batch_size=64, validation_data=(X_dev_pad, y_dev), callbacks=[early_stopping])

# Save the model after training
model.save('bidirectional_lstm_model.h5')



Epoch 1/10
371/371 [==============================] - 168s 441ms/step - loss: 0.4649 - accuracy: 0.7804 - val_loss: 0.4070 - val_accuracy: 0.8068
Epoch 2/10
371/371 [==============================] - 173s 466ms/step - loss: 0.3401 - accuracy: 0.8504 - val_loss: 0.4613 - val_accuracy: 0.7892
Epoch 3/10
371/371 [==============================] - 175s 472ms/step - loss: 0.2639 - accuracy: 0.8912 - val_loss: 0.4750 - val_accuracy: 0.8005
Epoch 4/10
371/371 [==============================] - 173s 466ms/step - loss: 0.1997 - accuracy: 0.9214 - val_loss: 0.5390 - val_accuracy: 0.7886


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(




---



## **EVALUATION ON DEVELOPMENT DATASET**

The following code should be run for evaluation on the entire development dataset. They can be run independently.

In [ ]:
import pandas as pd
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from keras.preprocessing.sequence import pad_sequences

import pickle

# Load the saved model
model_loaded = load_model('bidirectional_lstm_model.h5')
# load development dataset
dev_data = pd.read_csv('dev.csv')
X_dev = dev_data['Claim'] + " " + dev_data['Evidence']
y_dev = dev_data['label']

# Tokenize and pad sequences for the development data
# use the tokenizer that was fit on the training data
# Load the tokenizer from a file
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

max_len = 100  #this cover 100% of the dataset

X_dev_seq = tokenizer.texts_to_sequences(X_dev)
X_dev_pad = pad_sequences(X_dev_seq, maxlen=max_len)  # Ensure max_len is the same as for the training data


# Evaluate the model on the development/validation set
dev_loss, dev_accuracy = model_loaded.evaluate(X_dev_pad, y_dev, verbose=1)
print(f"Development Set Loss: {dev_loss}")
print(f"Development Set Accuracy: {dev_accuracy*100:.2f}%")


# Make predictions on the development set
dev_predictions = (model_loaded.predict(X_dev_pad) > 0.5).astype(int)

# Calculate and print metrics
accuracy = accuracy_score(y_dev, dev_predictions)
f1 = f1_score(y_dev, dev_predictions, average='weighted')
precision = precision_score(y_dev, dev_predictions, average='weighted')
recall = recall_score(y_dev, dev_predictions, average='weighted')

print(f"Accuracy on the development set: {accuracy:.4f}")
print(f"F1 Score on the development set: {f1:.4f}")
print(f"Precision on the development set: {precision:.4f}")
print(f"Recall on the development set: {recall:.4f}")
print("\nClassification Report:")
print(classification_report(y_dev, dev_predictions))

186/186 [==============================] - 22s 106ms/step - loss: 0.4062 - accuracy: 0.8102
Development Set Loss: 0.4061630666255951
Development Set Accuracy: 81.02%
186/186 [==============================] - 22s 112ms/step
Accuracy on the development set: 0.8102
F1 Score on the development set: 0.8060
Precision on the development set: 0.8040
Recall on the development set: 0.8102

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      4327
           1       0.67      0.59      0.62      1599

    accuracy                           0.81      5926
   macro avg       0.76      0.74      0.75      5926
weighted avg       0.80      0.81      0.81      5926





---



## **PRE-TESTING ON TRIAL DATASET**





This code is intended to further evaluate the model using a trial dataset. They can also be run independently.

In [ ]:
import pandas as pd
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from keras.preprocessing.sequence import pad_sequences

import pickle

# Load the saved model
model_loaded = load_model('bidirectional_lstm_model.h5')
# load trial dataset
trial_data = pd.read_csv('trial.csv')
X_trial = trial_data['Claim'] + " " + trial_data['Evidence']
y_trial = trial_data['label']

# Tokenize and pad sequences for the trial data
# use the tokenizer that was fit on the training data
# Load the tokenizer from a file
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

max_len = 100  #this cover 100% of the dataset

X_trial_seq = tokenizer.texts_to_sequences(X_trial)
X_trial_pad = pad_sequences(X_trial_seq, maxlen=max_len)  # Ensure max_len is the same as for the training data


# Evaluate the model on the terial set
trial_loss, trial_accuracy = model_loaded.evaluate(X_trial_pad, y_trial, verbose=1)
print(f"Trial Set Loss: {trial_loss}")
print(f"Trial Set Accuracy: {trial_accuracy*100:.2f}%")


# Make predictions on the trial set
trial_predictions = (model_loaded.predict(X_trial_pad) > 0.5).astype(int)

# Calculate and print metrics
accuracy = accuracy_score(y_trial, trial_predictions)
f1 = f1_score(y_trial, trial_predictions, average='weighted')
precision = precision_score(y_trial, trial_predictions, average='weighted')
recall = recall_score(y_trial, trial_predictions, average='weighted')

print(f"Accuracy on the Trial set: {accuracy:.4f}")
print(f"F1 Score on the Trial set: {f1:.4f}")
print(f"Precision on the Trial set: {precision:.4f}")
print(f"Recall on the Trial set: {recall:.4f}")
print("\nClassification Report:")
print(classification_report(y_trial, trial_predictions))

2/2 [==============================] - 1s 43ms/step - loss: 0.2758 - accuracy: 0.9000
Trial Set Loss: 0.27575358748435974
Trial Set Accuracy: 90.00%
2/2 [==============================] - 1s 39ms/step
Accuracy on the Trial set: 0.9000
F1 Score on the Trial set: 0.8990
Precision on the Trial set: 0.8988
Recall on the Trial set: 0.9000

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.94      0.93        35
           1       0.86      0.80      0.83        15

    accuracy                           0.90        50
   macro avg       0.89      0.87      0.88        50
weighted avg       0.90      0.90      0.90        50





---



# **DEMO CODE**





The following code is used to produce the prediction file.

In [ ]:
import pandas as pd
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle

# Load the saved model
model_loaded = load_model('bidirectional_lstm_model.h5')

# Load the test dataset
# To test with other dataset, upload the file into the environment first, and then replace 'test.csv' with other .csv file
test_data = pd.read_csv('test.csv')

# Ensure that 'Claim' and 'Evidence' columns are of string type
test_data['Claim'] = test_data['Claim'].astype(str)
test_data['Evidence'] = test_data['Evidence'].astype(str)

# Combine 'Claim' and 'Evidence' columns into a single text
X_test = test_data['Claim'] + " " + test_data['Evidence']

# Load the tokenizer from a file
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define max_len as the same value used during training
max_len = 100

# Tokenize and pad sequences for the test data
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Make predictions on the test set
test_predictions = model_loaded.predict(X_test_pad).round().astype(int)

# Create a DataFrame with the predictions
predictions_df = pd.DataFrame(test_predictions, columns=['prediction'])

# Save the predictions to a CSV file
predictions_df.to_csv('Group_56_B.csv', index=False)

print("Predictions have been written to Group_56_B.csv")


147/147 [==============================] - 18s 111ms/step
Predictions have been written to Group_56_B.csv
